In [1]:
import pandas as pd
import numpy as np
import os
import sys

import matplotlib as plt

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### For reproducibility, please make sure PUIDATA environmental variable is properly set up

In [2]:
PUIdata = os.getenv('PUIDATA')
print(PUIdata)

/home/cusp/uc288/PUIdata


In [3]:
path = PUIdata + '/ADSProject'

if (not os.path.isdir(path)):
    if (os.system('mkdir ' + path) == 0):
        print("OK, directory in place")
else:
    print("OK, directory in place")

OK, directory in place


### Downloading Annualized Rollings Sales for 2015 and 2016
Access all other files here: http://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page

http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2015/2015_manhattan.xls

In [4]:
boroughs = ['manhattan', 'bronx', 'brooklyn', 'queens', 'statenisland']

In [5]:
nyc2015 = pd.DataFrame()

In [6]:
nyc2016 = pd.DataFrame()

In [7]:
for boro in boroughs:
    filename = '2015_' + boro + '.xls'
    if not os.path.isfile(path + '/' + filename):
        print("File not found for " + filename)
        print("Download file for reading...")
        os.system('curl -O http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2015/' + filename)
        os.system('mv ' + filename + ' ' + path)
    
    if os.path.isfile(path + '/' + filename):
        print("File " + filename + " in place, read file")
        nyc2015 = nyc2015.append(pd.read_excel(path + '/' + filename, skiprows=4))
        
print(nyc2015.shape)

File 2015_manhattan.xls in place, read file
File 2015_bronx.xls in place, read file
File 2015_brooklyn.xls in place, read file
File 2015_queens.xls in place, read file
File 2015_statenisland.xls in place, read file
(91446, 21)


In [8]:
for boro in boroughs:
    filename = '2016_' + boro + '.xls'
    if not os.path.isfile(path + '/' + filename):
        print("File not found for " + filename)
        print("Download file for reading...")
        os.system('curl -O http://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/annualized-sales/2016/' + filename)
        os.system('mv ' + filename + ' ' + path)
    
    if os.path.isfile(path + '/' + filename):
        print("File " + filename + " in place, read file")
        nyc2016 = nyc2016.append(pd.read_excel(path + '/' + filename, skiprows=4))
        
print(nyc2016.shape)

File 2016_manhattan.xls in place, read file
File 2016_bronx.xls in place, read file
File 2016_brooklyn.xls in place, read file
File 2016_queens.xls in place, read file
File 2016_statenisland.xls in place, read file
(89806, 21)


In [9]:
def fixColumns(df):
    cols = df.columns.str.replace('\n', '').str.replace(' ', '_')
    df.columns = cols

In [10]:
def showDesc(df):
    print('Number of sales with $0 price: {}'.format(df[df['SALE_PRICE'] == 0]['LOT'].count()))
    print('Number of sales with negative price: {}'.format(df[df['SALE_PRICE'] < 0]['LOT'].count()))
    print('Number of sales with 0 year built: {}'.format(df[df['YEAR_BUILT'] == 0]['LOT'].count()))

In [11]:
def dropZero(df):
    return df[df['SALE_PRICE'] > 0]

In [12]:
fixColumns(nyc2015)
fixColumns(nyc2016)

In [13]:
print(nyc2015.shape)
nyc2015 = dropZero(nyc2015)
print(nyc2015.shape)

(91446, 21)
(64392, 21)


In [14]:
print(nyc2016.shape)
nyc2016 = dropZero(nyc2016)
print(nyc2016.shape)

(89806, 21)
(62195, 21)


In [15]:
tax_class = ['1', '1A', '1C', '2', '2A', '2B', '2C']

In [16]:
def convertToStr(series):
    return series.astype(str)

In [17]:
def getResidential(df):
    return df[df['TAX_CLASS_AT_PRESENT'].isin(tax_class)]

In [18]:
nyc2015['TAX_CLASS_AT_PRESENT'] = convertToStr(nyc2015['TAX_CLASS_AT_PRESENT'])
nyc2016['TAX_CLASS_AT_PRESENT'] = convertToStr(nyc2016['TAX_CLASS_AT_PRESENT'])

In [19]:
print(nyc2015.shape)
nyc2015 = getResidential(nyc2015)
print(nyc2015.shape)

(64392, 21)
(57607, 21)


In [20]:
print(nyc2016.shape)
nyc2016 = getResidential(nyc2016)
print(nyc2016.shape)

(62195, 21)
(56003, 21)


This field consists of the borough code followed by the tax block followed by
the tax lot. The borough code is one numeric digit. The tax block is one to
five numeric digits, preceded with leading zeros when the block is less than
five digits. The tax lot is one to four digits and is preceded with leading zeros
when the lot is less than four digits.

In [21]:
def createBBL(df):
    df['BBL'] = df['BOROUGH'].astype(str).str.cat(df['BLOCK'].astype(str).str.pad(5, fillchar='0')).str.cat(df['LOT'].astype(str).str.pad(4, fillchar='0'))
    return df

In [22]:
nyc2015 = createBBL(nyc2015)
nyc2015.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE', 'BBL'],
      dtype='object')

In [23]:
nyc2016 = createBBL(nyc2016)
nyc2016.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'APARTMENT_NUMBER', 'ZIP_CODE',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE', 'BBL'],
      dtype='object')

In [24]:
nycsales2015 = nyc2015.groupby(['BBL', 'ZIP_CODE', 'NEIGHBORHOOD'])['SALE_PRICE'].median().reset_index()
nycsales2015['YEAR'] = 2015
nycsales2015.head()

,BBL,ZIP_CODE,NEIGHBORHOOD,SALE_PRICE,YEAR
0,1000151108,10004,FINANCIAL,1150000.0,2015
1,1000151136,10004,FINANCIAL,575000.0,2015
2,1000151160,10004,FINANCIAL,565000.0,2015
3,1000151170,10004,FINANCIAL,675000.0,2015
4,1000151177,10004,FINANCIAL,785000.0,2015


In [25]:
nycsales2016 = nyc2016.groupby(['BBL', 'ZIP_CODE', 'NEIGHBORHOOD'])['SALE_PRICE'].median().reset_index()
nycsales2016['YEAR'] = 2016
nycsales2016.head()

,BBL,ZIP_CODE,NEIGHBORHOOD,SALE_PRICE,YEAR
0,1000070038,10004,FINANCIAL,4027000.0,2016
1,1000110014,10004,FINANCIAL,470000.0,2016
2,1000151127,10004,FINANCIAL,600000.0,2016
3,1000151131,10004,FINANCIAL,45000.0,2016
4,1000151147,10004,FINANCIAL,595000.0,2016
